In [75]:
import pandas as pd 
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get credentials from environment variables
db_user = os.getenv("db_user")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")
db_name = os.getenv("db_name")

# Create the database engine
engine = create_engine(f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}')

In [76]:
query = "SELECT * FROM customers;"
customers_df = pd.read_sql(query, engine)
customers_df.head()  # Fetches all data from the 'customers' table

,customer_id,name,email,phone,address,date_of_birth,gender,country,city,registration_date,loyalty_points
0,CST000021,Lisa Roberts,lisa.roberts@yahoo.com,+590718260821,"1570 Carter Corner Suite 756, Butlermouth, CA ...",1972-04-04,Non-binary,"Palestine, State of",Lake Christineberg,2022-03-17,483
1,CST000027,Tracy Stark,tracy.stark@outlook.com,+508973504027,"506 Garrett Mission Apt. 785, New Benjaminshir...",1971-01-09,Non-binary,Bahamas,Yoderland,2021-02-05,577
2,CST000042,Gary Phillips,gary.phillips@gmail.com,+593423797775,"017 Amy Crossing Suite 580, Port Erikland, WV ...",1972-05-01,Non-binary,Seychelles,Port Barbara,2023-12-08,162
3,CST000045,Russell Orozco,russell.orozco@aol.com,+886165316048,"6104 Reynolds Street, New Madisonside, NY 83553",1968-05-30,Female,Uruguay,North Heatherville,2023-08-10,627
4,CST000046,Hannah Powell,hannah.powell@gmail.com,+91700233971,"48818 West Prairie Suite 145, South Shane, VA ...",1996-12-14,Female,Mauritius,Williamsville,2021-09-25,907


In [4]:
query = "SELECT COUNT(*) AS order_count FROM orders;"
order_count = pd.read_sql(query, engine)
order_count  # Counts the total number of orders in the 'orders' table

,order_count
0,181366


In [5]:
query = "SELECT * FROM products WHERE category_id = (SELECT category_id FROM categories WHERE name = 'Electronics');"
electronics_df = pd.read_sql(query, engine)
electronics_df.head()  # Fetches all products belonging to the 'Electronics' category

,product_id,product_name,category_id,price,stock_quantity,supplier_id,warranty_period,country_of_origin
0,PRO000091,To Home,8,227.92,76,81660,Lifetime,Montserrat
1,PRO000092,Consumer Live,8,255.83,9,175810,3 years,Yemen
2,PRO000096,Material Theory,8,335.97,0,67313,3 years,Zambia
3,PRO000125,For Green,8,108.33,45,35874,Lifetime,Iraq
4,PRO000156,Card If,8,179.93,60,125949,1 year,Sri Lanka


In [6]:
query = """
SELECT o.order_id, o.order_date, o.order_amount, c.name AS customer_name
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id;
"""
orders_with_customers = pd.read_sql(query, engine)
orders_with_customers.head()  # Joins 'orders' and 'customers' to get order details and customer names

,order_id,order_date,order_amount,customer_name
0,ORD000001,2024-04-05,448.12,Andrea Hess
1,ORD000005,2024-10-25,547.01,Kayla Le
2,ORD000009,2024-08-21,91.98,Rebecca Norman
3,ORD000021,2025-04-12,314.03,Christina Hickman
4,ORD000026,2021-06-20,48.75,Mark Guerrero


In [7]:
query = """
SELECT o.customer_id, SUM(o.order_amount) AS total_sales
FROM orders o
GROUP BY o.customer_id;
"""
sales_per_customer = pd.read_sql(query, engine)
sales_per_customer.head()  # Summarizes total sales per customer

,customer_id,total_sales
0,CST000021,474.45
1,CST000027,461.04
2,CST000042,427.86
3,CST000045,743.56
4,CST000046,674.36


In [8]:
query = """
SELECT p.product_name, c.name AS category_name, s.name AS supplier_name
FROM products p
JOIN categories c ON p.category_id = c.category_id
JOIN suppliers s ON p.supplier_id = s.supplier_id;
"""
products_info = pd.read_sql(query, engine)
products_info.head()  # Retrieves product names along with category and supplier information

,product_name,category_name,supplier_name
0,Sit Town,Automotive,"Sanford, Peterson and Williams"
1,Military South,Automotive,Anderson-Erickson
2,Help Modern,Automotive,"Jackson, Bowen and May"
3,Rate Camera,Automotive,"Frye, Vega and Johnson"
4,Make Tell,Automotive,"Reyes, Rodriguez and Wells"


In [9]:
query = """
SELECT o.shipper_name, AVG(o.order_amount) AS avg_order_amount
FROM orders o
GROUP BY o.shipper_name;
"""
avg_order_per_shipper = pd.read_sql(query, engine)
avg_order_per_shipper.head()  # Calculates the average order amount for each shipper

,shipper_name,avg_order_amount
0,Abbott and Sons,536.670000
1,Abbott Group,564.006364
2,Abbott Inc,476.444286
3,Abbott LLC,365.705000
4,Abbott Ltd,552.897143


In [10]:
query = """
SELECT o.order_id, o.order_date, p.payment_status
FROM orders o
LEFT JOIN payments p ON o.order_id = p.order_id
WHERE p.payment_status IS NULL OR p.payment_status = 'Pending';
"""
pending_payments = pd.read_sql(query, engine)
pending_payments.head()  # Fetches orders that haven't been paid yet or are marked as 'Pending'

,order_id,order_date,payment_status
0,ORD000005,2024-10-25,Pending
1,ORD000021,2025-04-12,Pending
2,ORD000027,2024-02-28,Pending
3,ORD000072,2024-12-12,Pending
4,ORD000074,2023-07-22,Pending


In [11]:
query = """
SELECT p.product_name, SUM(oi.quantity * oi.unit_price) AS total_sales
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_name
ORDER BY total_sales DESC
LIMIT 5;
"""
top_selling_products = pd.read_sql(query, engine)
top_selling_products.head()  # Identifies the top 5 best-selling products based on sales volume

,product_name,total_sales
0,Body Management,7513.77
1,Art Purpose,7192.03
2,Admit East,7177.68
3,Become Shoulder,6867.48
4,Citizen Remain,6658.65


In [12]:
query = """
SELECT c.name AS category_name, SUM(oi.quantity * oi.unit_price) AS total_sales
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN categories c ON p.category_id = c.category_id
WHERE oi.order_id IN (SELECT order_id FROM orders WHERE order_date BETWEEN '2023-01-01' AND '2023-12-31')
GROUP BY c.name;
"""
sales_by_category = pd.read_sql(query, engine)
sales_by_category.head()  # Summarizes total sales by category for orders made in 2023

,category_name,total_sales
0,Sports,3179573.13
1,Furniture,3197176.07
2,Home,3142059.18
3,Books,3202019.51
4,Clothing,3294374.50


In [13]:
query = """
SELECT o.order_id, o.order_date, o.order_amount,
       SUM(o.order_amount) OVER (ORDER BY o.order_date) AS running_total
FROM orders o;
"""
running_total_sales = pd.read_sql(query, engine)
running_total_sales.head()  # Shows the running total of order amounts

,order_id,order_date,order_amount,running_total
0,ORD522330,2020-04-20,923.25,923.25
1,ORD923509,2020-04-24,470.63,1414.54
2,ORD380550,2020-04-24,20.66,1414.54
3,ORD658190,2020-04-25,624.08,2038.62
4,ORD549946,2020-04-27,780.26,2818.88


In [14]:
query = """
SELECT o.customer_id, MIN(o.order_date) AS first_purchase,
       MAX(o.order_date) AS last_purchase
FROM orders o
GROUP BY o.customer_id;
"""
first_last_purchase = pd.read_sql(query, engine)
first_last_purchase.head()  # Finds the first and last purchase date for each customer

,customer_id,first_purchase,last_purchase
0,CST000021,2024-03-27,2024-03-27
1,CST000027,2024-01-19,2024-01-19
2,CST000042,2024-04-27,2024-04-27
3,CST000045,2024-05-02,2024-05-02
4,CST000046,2022-05-12,2023-01-07


In [15]:
query = """
SELECT oi.order_id, p.product_name, SUM(oi.quantity) AS total_quantity,
       AVG(oi.quantity) AS avg_quantity
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY oi.order_id, p.product_name;
"""
avg_total_quantity = pd.read_sql(query, engine)
avg_total_quantity.head()  # Shows total and average quantity sold for each product per order

,order_id,product_name,total_quantity,avg_quantity
0,ORD000001,Popular Wear,5.0,5.0
1,ORD000005,Into Line,3.0,3.0
2,ORD000009,Tree Sometimes,4.0,4.0
3,ORD000021,Expert Eight,3.0,3.0
4,ORD000026,However Remain,2.0,2.0


In [16]:
query = """
SELECT c.customer_id, c.name, c.email
FROM customers c
WHERE c.customer_id IN (
    SELECT o.customer_id
    FROM orders o
    WHERE o.order_amount > 1000
);
"""
high_value_customers = pd.read_sql(query, engine)
high_value_customers.head()  # Finds customers who have made orders above 1000

,customer_id,name,email


In [17]:
query = """
SELECT c.customer_id, c.name, o.order_id, o.order_date
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_date = (
    SELECT MAX(order_date)
    FROM orders
    WHERE customer_id = c.customer_id
);
"""
most_recent_orders = pd.read_sql(query, engine)
most_recent_orders.head()  # Retrieves the most recent order for each customer

,customer_id,name,order_id,order_date
0,CST012874,Andrea Hess,ORD000001,2024-04-05
1,CST720312,Kayla Le,ORD000005,2024-10-25
2,CST653526,Rebecca Norman,ORD000009,2024-08-21
3,CST926010,Christina Hickman,ORD000021,2025-04-12
4,CST681862,Mark Guerrero,ORD000026,2021-06-20


In [18]:
query = """
SELECT s.shipper_name, SUM(o.shipping_cost) AS total_shipping_cost
FROM orders o
JOIN shippers s ON o.shipper_name = s.shipper_name
GROUP BY s.shipper_name
ORDER BY total_shipping_cost DESC
LIMIT 3;
"""
top_shippers = pd.read_sql(query, engine)
top_shippers.head()  # Fetches top 3 shippers based on total shipping cost

,shipper_name,total_shipping_cost
0,Smith Inc,6858.33
1,Smith PLC,6563.85
2,Smith and Sons,6477.44


In [19]:
query = """
SELECT p.payment_method, AVG(p.amount) AS avg_payment_amount
FROM payments p
GROUP BY p.payment_method;
"""
avg_payment_per_method = pd.read_sql(query, engine)
avg_payment_per_method.head()  # Calculates average payment amount by payment method

,payment_method,avg_payment_amount
0,PayPal,737.966449
1,Bank Transfer,746.956255
2,Credit Card,744.480880
3,Mobile Money,741.503573


In [20]:
query = """
SELECT p.product_name, p.stock_quantity, SUM(oi.quantity) AS total_sold
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_name, p.stock_quantity
HAVING p.stock_quantity - COALESCE(SUM(oi.quantity), 0) <= 10
ORDER BY total_sold DESC;
"""

low_stock_products = pd.read_sql(query, engine)
low_stock_products.head()  # products with low stock (<= 10) based on total quantity sold

,product_name,stock_quantity,total_sold
0,Court Manage,13,17.0
1,Above That,8,16.0
2,Administration Husband,9,14.0
3,Administration Try,12,14.0
4,Benefit Light,11,14.0


In [21]:
query = """
SELECT c.customer_id, c.name
FROM customers c
WHERE c.customer_id NOT IN (
    SELECT o.customer_id
    FROM orders o
    WHERE o.order_date >= CURDATE() - INTERVAL 1 YEAR
);
"""
inactive_customers = pd.read_sql(query, engine)
inactive_customers.head()  # Finds customers who haven't placed orders in the last year

,customer_id,name
0,CST000021,Lisa Roberts
1,CST000027,Tracy Stark
2,CST000046,Hannah Powell
3,CST000064,Paul George
4,CST000073,Daniel Miller


In [22]:
query = """
SELECT o.customer_id, COUNT(o.order_id) AS num_orders
FROM orders o
GROUP BY o.customer_id
ORDER BY num_orders DESC
LIMIT 5;
"""
top_customers_by_orders = pd.read_sql(query, engine)
top_customers_by_orders.head()  # Identifies the top 5 customers based on the number of orders

,customer_id,num_orders
0,CST068020,5
1,CST912561,5
2,CST903221,5
3,CST111517,4
4,CST176720,4


In [23]:
query = """
SELECT YEAR(o.order_date) AS order_year, COUNT(o.order_id) AS num_orders, SUM(o.order_amount) AS total_sales
FROM orders o
GROUP BY YEAR(o.order_date)
ORDER BY order_year;
"""
sales_by_year = pd.read_sql(query, engine)
sales_by_year.head()  # Aggregates number of orders and total sales by year

,order_year,num_orders,total_sales
0,2020,1873,943882.57
1,2021,10225,5174415.13
2,2022,21173,10822209.34
3,2023,37769,19257682.39
4,2024,69795,35375523.84


In [24]:
query = """
SELECT p.product_name, COUNT(DISTINCT p.supplier_id) AS num_suppliers
FROM products p
GROUP BY p.product_name
HAVING num_suppliers > 1;
"""
multiple_suppliers_products = pd.read_sql(query, engine)
multiple_suppliers_products.head()  # Identifies products supplied by more than one supplier

,product_name,num_suppliers
0,A Agency,2
1,A Apply,2
2,A Become,2
3,A Case,2
4,A Design,2


In [25]:
query = """
SELECT s.shipper_name, SUM(o.order_amount) AS total_order_value
FROM orders o
JOIN shippers s ON o.shipper_name = s.shipper_name
GROUP BY s.shipper_name
ORDER BY total_order_value DESC
LIMIT 3;
"""
top_shippers = pd.read_sql(query, engine)
top_shippers.head()  # Fetches top 3 shippers based on total order value

,shipper_name,total_order_value
0,Smith Inc,132506.64
1,Smith Ltd,119870.61
2,Smith and Sons,112667.90


In [26]:
query = """
SELECT c.customer_id, c.name, COUNT(o.order_id) AS order_count,
       CASE
           WHEN COUNT(o.order_id) > 20 THEN 'High Frequency'
           WHEN COUNT(o.order_id) BETWEEN 10 AND 20 THEN 'Medium Frequency'
           ELSE 'Low Frequency'
       END AS frequency_category
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name
ORDER BY order_count DESC;
"""
customer_segments = pd.read_sql(query, engine)
customer_segments.head()  # Classifies customers into high, medium, low frequency

,customer_id,name,order_count,frequency_category
0,CST912561,James Miller,5,Low Frequency
1,CST903221,Amanda Rowland,5,Low Frequency
2,CST068020,Dave Walter,5,Low Frequency
3,CST073364,Brandon Figueroa,4,Low Frequency
4,CST088677,Margaret Taylor,4,Low Frequency


In [27]:
query = """
SELECT c.country, c.name, AVG(o.order_amount) AS avg_order_value
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.country, c.customer_id
ORDER BY avg_order_value DESC;
"""
avg_order_per_country = pd.read_sql(query, engine)
avg_order_per_country.head()  # Average order value for each customer by country

,country,name,avg_order_value
0,Denmark,Maria Long,1000.00
1,Dominican Republic,Paula Gonzalez,1000.00
2,Cayman Islands,Joseph Williams,1000.00
3,Aruba,Daniel Snow,1000.00
4,Andorra,Robin Park,999.99


In [28]:
query = """
SELECT YEAR(o.order_date) AS year, MONTH(o.order_date) AS month,
       SUM(o.order_amount) AS total_sales, SUM(o.shipping_cost) AS total_shipping_cost
FROM orders o
GROUP BY YEAR(o.order_date), MONTH(o.order_date)
ORDER BY year DESC, month DESC;
"""
monthly_sales_shipping = pd.read_sql(query, engine)
monthly_sales_shipping.head()  # Total sales and shipping cost per month

,year,month,total_sales,total_shipping_cost
0,2025,4,4812449.41,262256.33
1,2025,3,6256088.80,340832.36
2,2025,2,4849814.50,261374.87
3,2025,1,4686842.53,253880.91
4,2024,12,4271227.13,229980.77


In [29]:
query = """
SELECT c.customer_id, c.name, SUM(o.order_amount) AS lifetime_value
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name
ORDER BY lifetime_value DESC
LIMIT 5;
"""
top_customers_lifetime_value = pd.read_sql(query, engine)
top_customers_lifetime_value.head()  # Top 5 customers based on lifetime value

,customer_id,name,lifetime_value
0,CST448047,Darren Trevino,3069.94
1,CST819583,James Rhodes,2940.68
2,CST225773,Alyssa Chambers,2939.29
3,CST797874,Brendan Smith,2868.35
4,CST073364,Brandon Figueroa,2854.74


In [30]:
query = """
SELECT s.shipper_name, COUNT(o.order_id) AS order_count, AVG(o.shipping_cost) AS avg_shipping_cost
FROM orders o
JOIN shippers s ON o.shipper_name = s.shipper_name
GROUP BY s.shipper_name
ORDER BY order_count DESC;
"""
shipper_order_volume = pd.read_sql(query, engine)
shipper_order_volume.head()  # Order volume and average shipping cost by shipper

,shipper_name,order_count,avg_shipping_cost
0,Smith Inc,249,27.543494
1,Smith Ltd,232,25.392500
2,Smith and Sons,228,28.409825
3,Smith PLC,217,30.248157
4,Smith Group,206,26.054806


In [31]:
query = """
SELECT o.order_id, SUM(o.order_amount) AS total_revenue, p.payment_status AS payment_status
FROM orders o
JOIN payments p ON o.order_id = p.order_id
GROUP BY o.order_id, p.payment_status
ORDER BY total_revenue DESC;
"""
order_payment_status = pd.read_sql(query, engine)
order_payment_status.head()  # Revenue and payment status for each order

,order_id,total_revenue,payment_status
0,ORD647862,2983.62,Failed
1,ORD047417,2981.49,Completed
2,ORD077946,2973.63,Completed
3,ORD770393,2959.98,Pending
4,ORD773965,2883.87,Pending


In [32]:
query = """
SELECT p.product_id, p.product_name, SUM(oi.quantity) AS total_quantity_sold
FROM products p
JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id, p.product_name
ORDER BY total_quantity_sold DESC
LIMIT 5;
"""
top_selling_products = pd.read_sql(query, engine)
top_selling_products.head()

,product_id,product_name,total_quantity_sold
0,PRO233708,Court Manage,17.0
1,PRO885338,Above That,16.0
2,PRO057530,Doctor Money,16.0
3,PRO037188,Coach Claim,16.0
4,PRO385489,At Instead,15.0


In [33]:
query = """
SELECT o.customer_id, c.name, AVG(DATEDIFF(p.payment_date, o.order_date)) AS avg_payment_time
FROM orders o
JOIN payments p ON o.order_id = p.order_id
JOIN customers c ON o.customer_id = c.customer_id
GROUP BY o.customer_id, c.name
ORDER BY avg_payment_time;
"""
avg_payment_time_per_customer = pd.read_sql(query, engine)
avg_payment_time_per_customer.head()  # Average payment time per customer

,customer_id,name,avg_payment_time
0,CST521108,Isaiah Bond,-1304.0
1,CST352949,Patrick Perez,-1207.0
2,CST612197,Mario Wells,-1197.0
3,CST784510,Christina Gross,-1171.0
4,CST206781,Anna Johnson,-1169.0


In [34]:
query = """
SELECT customers.name, orders.order_id, orders.order_date, order_items.quantity, products.product_name AS product_name
FROM customers
INNER JOIN orders ON customers.customer_id = orders.customer_id
INNER JOIN order_items ON orders.order_id = order_items.order_id
INNER JOIN products ON order_items.product_id = products.product_id
LIMIT 10;
"""

# Execute the query and fetch the result into a DataFrame
df = pd.read_sql(query, engine)

# Display the result
df.head()

,name,order_id,order_date,quantity,product_name
0,Derek Frazier,ORD044555,2024-12-27,4,Subject Than
1,Lauren Roberts,ORD505152,2025-03-03,4,Whole Though
2,Jody Barrera,ORD360099,2022-11-27,1,Reason Significant
3,Michael Fernandez,ORD395890,2024-08-12,2,Yourself Key
4,Shane Watkins,ORD892743,2025-02-15,4,Director Effort


In [35]:
query = """
SELECT customers.name, COUNT(orders.order_id) AS total_orders, SUM(order_items.quantity * order_items.unit_price) AS total_spent
FROM customers
LEFT JOIN orders ON customers.customer_id = orders.customer_id
LEFT JOIN order_items ON orders.order_id = order_items.order_id
GROUP BY customers.customer_id;
"""

df = pd.read_sql(query, engine)
df.head()

,name,total_orders,total_spent
0,Lisa Roberts,1,NaN
1,Tracy Stark,1,919.59
2,Gary Phillips,1,1845.20
3,Russell Orozco,2,1390.45
4,Hannah Powell,2,1387.22


In [36]:
query = """
SELECT shippers.shipper_name, COUNT(orders.order_id) AS total_orders
FROM shippers
RIGHT JOIN orders ON shippers.shipping_address = orders.shipping_address
WHERE orders.order_status = 'shipped'
GROUP BY shippers.shipper_name;
"""

df = pd.read_sql(query, engine)
df.head()


,shipper_name,total_orders
0,Adams-Roman,1
1,Adkins-Atkinson,1
2,"Allen, Baker and Jones",1
3,"Adams, Bell and Thomas",1
4,Alvarez-Mendez,1


In [38]:
# Customer Lifetime Value (CLV)
query = """
SELECT 
    c.customer_id,
    c.name AS customer_name,
    ROUND(SUM(p.amount), 2) AS lifetime_value
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN payments p ON o.order_id = p.order_id
GROUP BY c.customer_id
ORDER BY lifetime_value DESC
LIMIT 10;
"""

df = pd.read_sql(query, engine)
df.head()

,customer_id,customer_name,lifetime_value
0,CST261733,Andrea Shelton,7365.33
1,CST031928,Danny Rodriguez,7095.57
2,CST234478,Steven Martin,6991.13
3,CST615149,Gina Johnson,6772.96
4,CST089658,Anthony Richardson,6371.68


In [41]:
# Monthly Sales Trend
query = """
SELECT 
    DATE_FORMAT(order_date, '%Y-%m') AS month,
    ROUND(SUM(order_amount), 2) AS total_sales
FROM orders
WHERE order_date >= CURDATE() - INTERVAL 12 MONTH
GROUP BY month
ORDER BY month;
"""

df = pd.read_sql(query, engine)
df.head()

,month,total_sales
0,2024-04,1092032.25
1,2024-05,2665759.04
2,2024-06,2705548.91
3,2024-07,2958829.35
4,2024-08,3202634.09


In [42]:
# Top Loyal Customers by Order Frequency and Spend 
query = """
SELECT 
    c.customer_id,
    c.name AS customer_name,
    COUNT(o.order_id) AS total_orders,
    ROUND(SUM(p.amount), 2) AS total_spent,
    c.loyalty_points
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN payments p ON o.order_id = p.order_id
GROUP BY c.customer_id
ORDER BY total_orders DESC, total_spent DESC
LIMIT 10;
"""

df = pd.read_sql(query, engine)
df.head()

,customer_id,customer_name,total_orders,total_spent,loyalty_points
0,CST505525,George Davis,7,2824.80,52
1,CST031928,Danny Rodriguez,6,7095.57,503
2,CST341350,Angela Hartman,6,4496.72,154
3,CST046032,Alexander Scott,6,4231.87,180
4,CST066531,Edwin Harris,6,3541.44,998


In [44]:
# Most Frequently Purchased Products
query = """
-- If the correct column is `product_name`
SELECT 
    p.product_id,
    p.product_name AS product_name,
    COUNT(oi.order_item_id) AS total_purchases
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_id, p.product_name
ORDER BY total_purchases DESC
LIMIT 10;
"""

df = pd.read_sql(query, engine)
df.head()

,product_id,product_name,total_purchases
0,PRO233708,Court Manage,5
1,PRO634247,Find Left,5
2,PRO057530,Doctor Money,4
3,PRO072182,Area Minute,4
4,PRO025810,Doctor If,4


In [70]:
# Order Fulfillment Time (Speed of Delivery)
query = """
SELECT 
    o.order_id,
    o.order_date,
    p.payment_date,
    DATEDIFF(p.payment_date, o.order_date) AS fulfillment_time_days
FROM 
    orders o
JOIN 
    payments p ON o.order_id = p.order_id
WHERE 
    DATEDIFF(p.payment_date, o.order_date) >= 0
ORDER BY 
    fulfillment_time_days DESC
LIMIT 10;
"""

df = pd.read_sql(query, engine)
df.head()

,order_id,order_date,payment_date,fulfillment_time_days
0,ORD247739,2020-05-09,2025-04-17,1804
1,ORD328427,2020-05-13,2025-04-13,1796
2,ORD121243,2020-05-08,2025-04-06,1794
3,ORD152445,2020-05-06,2025-03-29,1788
4,ORD287326,2020-06-14,2025-04-11,1762


In [46]:
# Order Fulfillment Time (Speed of Delivery)
query = """
SELECT 
    c.country,
    c.city,
    ROUND(SUM(o.order_amount), 2) AS total_sales
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.country, c.city
ORDER BY total_sales DESC
LIMIT 15;
"""

df = pd.read_sql(query, engine)
df.head()

,country,city,total_sales
0,Slovakia,North Michael,3508.20
1,Bhutan,Brandonton,3397.80
2,Trinidad and Tobago,Scottmouth,3298.16
3,Malawi,Port Scott,3271.78
4,Antarctica,South Robert,3200.34


In [47]:
# Sales by Country and City

query = """
SELECT 
    c.country,
    c.city,
    ROUND(SUM(o.order_amount), 2) AS total_sales
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.country, c.city
ORDER BY total_sales DESC
LIMIT 15;
"""

df = pd.read_sql(query, engine)
df.head()

,country,city,total_sales
0,Slovakia,North Michael,3508.20
1,Bhutan,Brandonton,3397.80
2,Trinidad and Tobago,Scottmouth,3298.16
3,Malawi,Port Scott,3271.78
4,Antarctica,South Robert,3200.34


In [ ]:
# Average Order Value (AOV) Per Customer
query = """
SELECT 
    c.customer_id,
    c.name AS customer_name,
    ROUND(AVG(o.order_amount), 2) AS avg_order_value,
    COUNT(o.order_id) AS total_orders
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id
HAVING total_orders >= 3
ORDER BY avg_order_value DESC
LIMIT 10;
"""

df = pd.read_sql(query, engine)
df.head()

In [59]:
# Average Order Value (AOV) Per Customer
query = """
SELECT 
    c.customer_id,
    c.name AS customer_name,
    ROUND(AVG(o.order_amount), 2) AS avg_order_value,
    COUNT(o.order_id) AS total_orders
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id
HAVING total_orders >= 3
ORDER BY avg_order_value DESC
LIMIT 10;
"""

df = pd.read_sql(query, engine)
df.head()

,customer_id,customer_name,avg_order_value,total_orders
0,CST819583,James Rhodes,980.23,3
1,CST797874,Brendan Smith,956.12,3
2,CST038384,Ann Walker,933.69,3
3,CST145437,Angela Rodgers,932.85,3
4,CST581348,Daniel Perkins,915.60,3


In [60]:
# High-Value Orders with Large Discounts
query = """
SELECT 
    oi.order_id,
    c.name AS customer_name,
    ROUND(SUM(oi.unit_price * oi.quantity), 2) AS gross_value,
    ROUND(SUM(oi.discount), 2) AS total_discount,
    ROUND(SUM(oi.discount) / SUM(oi.unit_price * oi.quantity) * 100, 2) AS discount_percent
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN customers c ON o.customer_id = c.customer_id
GROUP BY oi.order_id
HAVING discount_percent > 20
ORDER BY total_discount DESC
LIMIT 10;
"""

df = pd.read_sql(query, engine)
df.head()

,order_id,customer_name,gross_value,total_discount,discount_percent
0,ORD863068,Denise Aguilar,372.84,115.96,31.10
1,ORD370232,Douglas Brown,424.83,111.41,26.22
2,ORD705351,Miguel Mckenzie,433.46,95.77,22.09
3,ORD790061,Johnny Johnson,198.85,95.26,47.91
4,ORD196392,Daryl Beck,463.08,94.74,20.46


In [62]:
# Inventory Turnover Rate per Product
query = """
SELECT 
    p.product_id,
    p.product_name,
    p.stock_quantity,
    SUM(oi.quantity) AS units_sold,
    ROUND(SUM(oi.quantity) / p.stock_quantity, 2) AS turnover_rate
FROM products p
JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id
HAVING stock_quantity > 0
ORDER BY turnover_rate DESC
LIMIT 10;
"""

df = pd.read_sql(query, engine)
df.head()

,product_id,product_name,stock_quantity,units_sold,turnover_rate
0,PRO322732,Old Effort,1,13.0,13.0
1,PRO136059,Blue Hot,1,11.0,11.0
2,PRO241172,Do Reflect,1,10.0,10.0
3,PRO249562,Consumer From,1,10.0,10.0
4,PRO300257,Across Especially,1,10.0,10.0
